## Leetcode 452: Minimum Number of Arrows to Burst Balloons
> There are some spherical balloons spread in two-dimensional space. For each balloon, provided input is the start and end coordinates of the horizontal diameter. Since it's horizontal, y-coordinates don't matter, and hence the x-coordinates of start and end of the diameter suffice. The start is always smaller than the end.  
> An arrow can be shot up exactly vertically from different points along the x-axis. A balloon with xstart and xend bursts by an arrow shot at x if xstart ≤ x ≤ xend. There is no limit to the number of arrows that can be shot. An arrow once shot keeps traveling up infinitely.

> Example:  
> points = [[10,16],[2,8],[1,6],[7,12]]  
> Output: 2  
> One way is to shoot one arrow for example at x = 6 (bursting the balloons [2,8] and [1,6]) and another arrow at x = 11 (bursting the other two balloons).


## Solution:
> Greedy algorithm.
> 1. Sort the interval by end point. 
> 2. From left to right, each time we add one balloon. If it is bursted, we can continue to the next balloon. Otherwise, we need an additional arrow to burst it. Put the arrow to the end point to burst as much balloons that haven't seen as possible.

In [4]:
def burst_balloons(points):
    sorted_p = sorted(points, key=lambda x: x[1])
    shots = []
    
    for p in sorted_p:
        if len(shots)==0:
            shots.append(p[1])
        elif shots[-1]<p[0]:
            shots.append(p[1])
    return len(shots)
        

In [6]:
burst_balloons([[2,3],[2,3]])

1

## Leetcode 435. Non-overlapping Intervals
> Given a collection of intervals, find the minimum number of intervals you need to remove to make the rest of the intervals non-overlapping.  
> Example:  
> Input: [[1,2],[2,3],[3,4],[1,3]]   
> Output: 1  
> Explaination: [1,3] can be removed and the rest of intervals are non-overlapping.

## Solution:
> Greedy, still sort according to the end points.  
> Each time we consider one more interval. If the interval overlaps with the current remaining intervals, we need to remove it. (check if new interval start point is less than current max point)

In [10]:
def eraseOverlapIntervals(intervals):
    """
    :type intervals: List[List[int]]
    :rtype: int
    """
    sorted_intervals = sorted(intervals, key=lambda x:x[1])
    current_max = None
    count = 0
    for interval in sorted_intervals:
        if current_max==None:
            current_max = interval[1]
        elif interval[0]<current_max: # has overlap
            count += 1
        else: # no overlap
            current_max = interval[1]
    return count

In [11]:
eraseOverlapIntervals([[1,2],[2,3],[3,4],[1,3]])

1

## Leetcode 56: Merge Intervals
> Given a collection of intervals, merge all overlapping intervals.  
> **Example:**    
> Input: intervals = [[1,3],[2,6],[10,12],[8,10],[15,18]]  
> Output: [[1,6],[8,12],[15,18]]  
> Explanation: Since intervals [1,3] and [2,6] overlaps, merge them into [1,6]. [10,12] and [8,10] overlaps, merge them into [8,12].

## Solution:
> Again, sort according to the end point. 
> Each time, consider one more interval from left to right. Maintain a list of combined intervals. According to the start of current interval, find its position in the interval array (binary search on all the end points of the combined intervals). Merge accordingly.

In [33]:
def merge_intervals(intervals):
    """
    :type intervals: List[List[int]]
    :rtype: List[List[int]]
    """
    
    sorted_intervals = sorted(intervals, key=lambda x:x[1])
    
    res = []
    for interval in sorted_intervals:
        if len(res) == 0:
            res.append(interval)
        else:
            left = 0
            right = len(res)-1
            while (left<=right):
                mid = int((right-left)/2)+left
                if res[mid][1] < interval[0]: 
                    left = mid+1
                else:
                    right = mid-1
            
            if right==len(res)-1: # res[-1][1]<interval[0]
                res.append(interval)
            else: # res[right+1][1]>=interval[0]
                new_interval = [min(res[right+1][0],interval[0]),interval[1]]
                res = res[0:right+1] + [new_interval]
    return res
    

In [34]:
merge_intervals([[2,3],[2,2],[3,3],[1,3],[5,7],[2,2],[4,6]])

[[1, 3], [4, 7]]

In [35]:
l=[[1,2],[2,3]]

In [21]:
l[0:0]

[]

## Leetcode 252: Meeting rooms
> Given an array of meeting time intervals consisting of start and end times $[[s1,e1],[s2,e2],...] (si < ei)$, determine if a person could attend all meetings.

> Example:  
> Input: $[[0, 30],[5, 10],[15, 20]]$  
> Output: False

## Solution:
> Sort according to end time. Then check if each interval has start time no larger than the previous end time. Time complexity O(nlogn).

In [36]:
def attend_all_meetings(intervals):
    sorted_intervals = sorted(intervals, key=lambda x: x[1])
    for i in range(1, len(sorted_intervals)):
        if sorted_intervals[i-1][1] > sorted_intervals[i][0]:
            return False
    return True

In [40]:
attend_all_meetings([[5, 10],[15, 20],[12,15]])

True

## Leetcode 253: Meeting rooms II
> Given an array of meeting time intervals consisting of start and end times $[[s1,e1],[s2,e2],...] (si < ei)$, find the minimum number of conference rooms required.  
> **Example:**  
> Input: $[[0, 30],[5, 10],[15, 20]]$  
> Output: 2


## Solution:
> **General solution framework**:
> 1. Sort intervals according to start position.  
> 2. Keep track of the rightmost overlapped intervals (any two of them overlap). At most times, we do not need to use an explicit set to store them. Instead, we just need to maintain several key parameters, e.g. the number of overlapping intervals (count), the minimum ending point among all overlapping intervals (minEnd), etc.  
> 3. Start from the left, we consider one more interval at a time. If the interval has overlapped with the rightmost overlapped intervals, add it to our overlapped intervals set (by checking if interval.start<minEnd). Otherwise, the intervals after this one will not overlap with previous set of overlapped intervals (since we sort by start point), hence we can reset the rightmost overlapped intervals by dropping a few intervals.

> **Specific for Meeting rooms problem:**
> 1. Sort intervals according to start position.
> 2. Keep track of the rightmost overlapped intervals by recording the minimum ending point (minEnd).
> 3. Consider a new interval I.  
> i) if I.start < minEnd, I is also overlapped with the rightmost overlapped interval set. Add it to the set and set minEnd = min(minEnd, I.end)  
> ii) if I.start>=minEnd, I is not overlapped with the rightmost overlapped interval set. We then remove all intervals in the set with end point no larger than I.start. Thus, for better efficiency of the algorithm, we can maintain a heap/balanced BST to store the end points of these rightmost overlapped intervals so that each deletion and insertion can be performed within O(logn) time.

In [44]:
import heapq as hq

In [46]:
hq.heappush?

Signature: hq.heappush(heap, item, /)
Docstring: Push item onto heap, maintaining the heap invariant.
Type:      builtin_function_or_method


In [49]:
def minimum_meeting_rooms(intervals):
    sorted_i = sorted(intervals, key = lambda x: x[0])
    h = [] # min heap to store the end points of rightmost overlapped intervals
    count = 0
    for interval in sorted_i:
        while len(h)>0 and h[0]<=interval[0]:
            hq.heappop(h)
        hq.heappush(h, interval[1])
        count = max(count, len(h))
    return count

In [54]:
minimum_meeting_rooms([[0, 30],[5, 10],[15, 20],[11,14],[6,13]])

3